In [1]:
'''
Husayn El Sharif
Process DSSAT model runs to produce spreadsheets with monthly (April -- October) climate data linked to crop yield and irrigation amount
'''

'\nHusayn El Sharif\nProcess DSSAT model runs to produce spreadsheets with monthly (April -- October) climate data linked to crop yield and irrigation amount\n'

In [2]:
# import pacakges

import numpy as np
import pandas as pd

import glob # for file listing

from datetime import datetime, date, timedelta # for doy and datetime operations

import copy


In [3]:
'''
Note: all crops rainfed/irrgigated have the same weatherfile data
'''

# listing of weatherfiles
weatherfiles_search_string = '00 Archive/03 DSSAT Runs/rainfed/corn/DSSAT_Output/W*.WTH'
weatherfiles_path_list = glob.glob(weatherfiles_search_string)

In [4]:
# process weatherfiles
monthly_weather_data_df_dict = {} # initialize

for exp_idx, current_wf_path in enumerate(weatherfiles_path_list):
    # exp name
    exp_name = 'EXP_{:03d}'.format(exp_idx+1)
    
    # import daily weather data
    current_wf_df = pd.read_csv(current_wf_path, skiprows=5, engine='python', sep='\s+', names=['DATE', 'SRAD', 'TMAX', 'TMIN', 'PRE', 'DCO2'])
    
    # remove DCO2
    #current_wf_df.drop(labels=['DCO2'], axis=1, inplace=True)
    #current_wf_df
    
    # get year of interest (first year in weatherfile + 1 year = year of interest, so if weatherfile starts with 1979, the year of interest is 1980. Weatherfiles include one year before and one year after the year of interest)
    year_of_interest = int(str(current_wf_df.iloc[0]['DATE'])[:4])+1
    
    # filter dataframe for year of interest
    current_wf_df =  current_wf_df[(current_wf_df['DATE'] > year_of_interest*1000) & (current_wf_df['DATE'] <= year_of_interest*1000+366)]
    current_wf_df.reset_index(inplace=True, drop=True)
    
    # insert year column
    current_wf_df.insert(0, 'Year', year_of_interest)
    
    # create datetime column
    doy = current_wf_df['DATE'] - current_wf_df['Year']*1000
    current_wf_df.insert(1, 'DateTime', pd.to_datetime(current_wf_df['Year']*1000 + doy, format='%Y%j') )
    
    # create tavg column
    current_wf_df.insert(6, 'TAVG', current_wf_df[['TMAX', 'TMIN']].mean(axis=1))
    
    # perform monthly aggregation (note: precipitation is the monthly sum of daily precipitation)
    current_wf_df = current_wf_df.resample(rule='ME', on='DateTime').agg({'Year':'min','SRAD':'mean', 'TMAX':'mean', 'TMIN':'mean', 'TAVG':'mean', 'PRE':'sum', 'DCO2':'mean'})
    
    # filter just growing season months (April - October)
    current_wf_df.reset_index(inplace=True) # recreate DateTime column
    current_wf_df = current_wf_df[(current_wf_df['DateTime'].dt.month >=4) & (current_wf_df['DateTime'].dt.month <=10)]
    current_wf_df.reset_index(inplace=True, drop=True)
    
    # insert month column
    current_wf_df.insert(2, 'Month', current_wf_df['DateTime'].dt.month)

    # save final dataframe to dictionary
    monthly_weather_data_df_dict[exp_name] = current_wf_df
    

In [5]:
# Crop and Water Management
crop_names = ['corn', 'cotton', 'peanut', 'soybean']
water_mgt = ['rainfed', 'irrigated']

In [6]:
# Include Yield and Irrigation with Monthly Weater data

for current_crop_name in crop_names:
    for current_water_mgt in water_mgt:
        
        print('Currently Processing: {} {}'.format(current_crop_name, current_water_mgt))

        # retrieve biweekly data
        biweekly_cropmodel_spreadsheet_path = '00 Archive/04 Crop Model Results Agg Spreadsheets/{}_{}.xlsx'.format(current_crop_name, current_water_mgt)
        biweekly_cropmodel_df_dict = pd.read_excel(biweekly_cropmodel_spreadsheet_path, sheet_name=None)
        exp_name_list = list(biweekly_cropmodel_df_dict.keys())

        # export path
        monthly_cropmodel_spreadsheet_path = '02 Crop Model Results Agg Spreadsheets (Monthly)/{}_{}.xlsx'.format(current_crop_name, current_water_mgt)

        # deep copy monthly_weather_data_df_dict
        monthly_weather_data_df_dict_copy = copy.deepcopy(monthly_weather_data_df_dict)

        # process each experiment
        for exp_name in exp_name_list:
            # retrieve crop yield and irrigation demand (from biweekly dataframe)
            crop_yield = biweekly_cropmodel_df_dict[exp_name].iloc[0]['YIELD']
            total_irrigation = biweekly_cropmodel_df_dict[exp_name].iloc[0]['TOTAL_IRR']
            # append yield and irrigation demand to growing season monthly weather dataframe
            monthly_weather_data_df_dict_copy[exp_name]['YIELD'] = pd.Series(biweekly_cropmodel_df_dict[exp_name].iloc[0]['YIELD'], index=[0])
            monthly_weather_data_df_dict_copy[exp_name]['TOTAL_IRR'] = pd.Series(biweekly_cropmodel_df_dict[exp_name].iloc[0]['TOTAL_IRR'], index=[0])

        # write to excel sheet
        with pd.ExcelWriter(monthly_cropmodel_spreadsheet_path) as writer:
            for key, val in monthly_weather_data_df_dict_copy.items():
                val.to_excel(writer, sheet_name=key, index=False)

Currently Processing: corn rainfed
Currently Processing: corn irrigated
Currently Processing: cotton rainfed
Currently Processing: cotton irrigated
Currently Processing: peanut rainfed
Currently Processing: peanut irrigated
Currently Processing: soybean rainfed
Currently Processing: soybean irrigated
